In [16]:
import os
import re
import base64
import pandas as pd
import plotly.express as px
import dash_leaflet as dl
from dash import Dash, dcc, html, dash_table, callback_context
from dash.dependencies import Input, Output, State
from pymongo import MongoClient
from CRUD import AnimalShelter

# Connect to MongoDB
shelter = AnimalShelter()
df = pd.DataFrame.from_records(shelter.getRecordCriteria({}))

# App setup
app = Dash(__name__, suppress_callback_exceptions=True)

# Logo image
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout
app.layout = html.Div([
    html.A(html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), height=250, width=251)), href='https://www.snhu.edu', target="_blank"),
    html.Center(html.B(html.H1("Tyler Hamilton' SNHU CS-340 Dashboard"))),
    html.Hr(),

    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'All'},
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster'}
        ],
        value='All'
    ),
    html.Hr(),

    html.Div([
        dcc.Input(id='input-name', type='text', placeholder='Animal Name'),
        dcc.Input(id='input-breed', type='text', placeholder='Breed'),
        html.Button("Add Entry", id="add-button", n_clicks=0),
        html.Button("Edit Entry", id="edit-button", n_clicks=0),
        html.Button("Delete Entry", id="delete-button", n_clicks=0),
        html.Div(id="action-output", style={'marginTop': '10px', 'color': 'green'})
    ]),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single",
        selected_rows=[],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex', 'justify-content': 'center'}, children=[
        html.Div(id='graph-id', className='col s12 m6', children=[
            dcc.Graph(id='pie-graph')
        ]),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

@app.callback(
    [Output("datatable-id", "data"),
     Output("datatable-id", "columns"),
     Output("action-output", "children")],
    [
        Input("filter-type", "value"),
        Input("add-button", "n_clicks"),
        Input("edit-button", "n_clicks"),
        Input("delete-button", "n_clicks")
    ],
    [
        State("input-name", "value"),
        State("input-breed", "value"),
        State("datatable-id", "selected_rows"),
        State("datatable-id", "data")
    ]
)
def update_and_crud(filter_type, add_clicks, edit_clicks, delete_clicks,
                    name, breed, selected_rows, table_data):
    ctx = callback_context
    triggered = ctx.triggered[0]["prop_id"].split(".")[0] if ctx.triggered else "filter-type"
    message = ""

    # Handle CRUD operations
    if triggered == "add-button":
        if not name or not breed:
            message = "Name and breed are required to add an entry."
        else:
            new_entry = {
                "name": name,
                "breed": breed,
                "age_upon_outcome_in_weeks": 52,
                "sex_upon_outcome": "Intact Male",
                "location_lat": 30.27,
                "location_long": -97.74
            }
            success = shelter.createRecord(new_entry)
            message = "Added entry." if success else "Failed to add."

    elif triggered == "edit-button":
        if not selected_rows:
            message = "Please select a row to edit."
        else:
            original = table_data[selected_rows[0]]
            update = {}
            if name: update["name"] = name
            if breed: update["breed"] = breed
            if update:
                success = shelter.updateRecord({"name": original["name"], "breed": original["breed"]}, update)
                message = "Record updated." if success else "Update failed."
            else:
                message = "No update values provided."

    elif triggered == "delete-button":
        if not selected_rows:
            message = "Please select a row to delete."
        else:
            original = table_data[selected_rows[0]]
            success = shelter.deleteRecord({"name": original["name"], "breed": original["breed"]})
            message = "Deleted successfully." if success else "Delete failed."

    # Handle filtering
    query = {}
    if filter_type == 'Water':
        query = {
            '$or': [{"breed": {'$regex': re.compile(b, re.IGNORECASE)}} for b in ["lab", "chesa", "newf"]],
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Mountain':
        query = {
            '$or': [{"breed": {'$regex': re.compile(b, re.IGNORECASE)}} for b in ["german", "mala", "old engilish", "husk", "rott"]],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'Disaster':
        query = {
            '$or': [{"breed": {'$regex': re.compile(b, re.IGNORECASE)}} for b in ["german", "golden", "blood", "dober", "rott"]],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    df_filtered = pd.DataFrame.from_records(shelter.getRecordCriteria(query))
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns]
    return df_filtered.to_dict("records"), columns, message

@app.callback(
    Output('pie-graph', 'figure'),
    Input('datatable-id', 'data')
)
def update_graphs(rows):
    dff = pd.DataFrame(rows)
    if dff.empty or 'breed' not in dff.columns:
        return px.pie(names=["No data"], values=[1], title="No breed data")
    return px.pie(dff, names='breed', title="Breed Distribution")

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_selected_rows')],
    [State('datatable-id', 'data')]
)
def update_map(selected_rows, data):
    default_marker = (30.75, -97.48)  # Austin Animal Center fallback
    if not selected_rows or not data:
        return [dl.Map(style={'width': '700px', 'height': '450px'}, center=default_marker, zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=default_marker, children=[
                dl.Tooltip("Austin Animal Center"),
                dl.Popup([html.H1("Austin"), html.P("Home base")])
            ])
        ])]

    try:
        selected = data[selected_rows[0]]
        lat = selected.get("location_lat")
        lon = selected.get("location_long")

        if lat is None or lon is None:
            raise ValueError("Missing coordinates.")

        marker = (float(lat), float(lon))
        return [dl.Map(style={'width': '700px', 'height': '450px'}, center=marker, zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=marker, children=[
                dl.Tooltip(selected.get('breed', 'Unknown')),
                dl.Popup([
                    html.H1(selected.get('name', 'Unnamed')),
                    html.P(selected.get('breed', 'Unknown'))
                ])
            ])
        ])]
    except Exception as e:
        print("Map fallback due to error:", e)
        return [dl.Map(style={'width': '700px', 'height': '450px'}, center=default_marker, zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=default_marker, children=[
                dl.Tooltip("Austin Animal Center"),
                dl.Popup([html.H1("Austin"), html.P("Home base")])
            ])
        ])]

if __name__ == '__main__':
    app.run(debug=True)

Map fallback due to error: Missing coordinates.
